# 1. Thư viện

In [1]:
import os
import json
import re
from docx import Document

# 2. Thực thi

## 2.1. Hàm đếm từ

In [2]:
# Hàm đếm số tokens đơn giản dựa trên khoảng trắng
def count_tokens_simple(text):
    return len(text.split())

## 2.2. Hàm trích xuất các từ khóa

In [3]:
# Hàm trích xuất từ khóa từ văn bản, loại bỏ dấu câu ngoại trừ "/" và "-"
def extract_keywords(text):
    text = re.sub(r'[^\w\s/-]', '', text)
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

In [4]:
# Hàm xử lý các từ khóa: chuyển về chữ thường, giữ nguyên số La Mã và từ khóa có "/" hoặc "-"
def process_keywords(keywords):
    result = []
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường
    return result

## 2.3. Hàm xử lý từ khóa cho các key có giá trị không null

In [5]:
def process_value_to_keywords(value):
    if value is None or value.strip() == "":
        return None
    keywords = extract_keywords(value)
    return process_keywords(keywords)

## 2.4. Hàm xử lý tên file

In [6]:
def process_file_name(file_name):
    parts = file_name.replace('.docx', '').split('_')

    if len(parts) != 8:
        return None  # Nếu file không hợp lệ
    
    stt = parts[0]
    loai_van_ban = parts[1]
    noi_ban_hanh = parts[2]
    so_hieu = parts[3].replace("-", "/")
    linhvuc_nganh = parts[4]
    ngay_ban_hanh = parts[5].replace("-", "/")
    ngay_hieu_luc = parts[6] if parts[6] == "Đã biết" else parts[6].replace("-", "/")
    chu_de = parts[7]
    
    # Trích xuất và xử lý từ khóa từ file name
    metadata_keywords = extract_keywords(f"{loai_van_ban} {noi_ban_hanh} {so_hieu} {linhvuc_nganh} {ngay_ban_hanh} {ngay_hieu_luc} {chu_de}")
    metadata_keywords = process_keywords(metadata_keywords)

    return {
        "stt": stt,
        "loai_van_ban": loai_van_ban,
        "loai_van_ban_Keywords": process_value_to_keywords(loai_van_ban),
        "noi_ban_hanh": noi_ban_hanh,
        "noi_ban_hanh_Keywords": process_value_to_keywords(noi_ban_hanh),
        "so_hieu": so_hieu,
        "linhvuc_nganh": linhvuc_nganh,
        "linhvuc_nganh_Keywords": process_value_to_keywords(linhvuc_nganh),
        "ngay_ban_hanh": ngay_ban_hanh,
        "ngay_hieu_luc": ngay_hieu_luc,
        "chu_de": chu_de,
        "chu_de_Keywords": process_value_to_keywords(chu_de),
        "key_words": metadata_keywords  # Trả về từ khóa đã xử lý
    }

## 2.5. Hàm đọc và xử lý nội dung của file .docx

In [7]:
def process_docx(file_path, file_metadata, max_tokens):
    document = Document(file_path)
    
    current_chapter = None
    current_section = None
    current_mini_section = None
    data = []
    
    for i in range(len(document.paragraphs)):
        paragraph_text = document.paragraphs[i].text.strip()
        
        if paragraph_text.startswith("Chương"):
            current_chapter = paragraph_text
            current_section = None
            current_mini_section = None
            if i + 1 < len(document.paragraphs):
                next_paragraph = document.paragraphs[i + 1].text.strip()
                if not next_paragraph.startswith(("Mục", "Tiểu mục", "Điều")):
                    current_chapter += f": {next_paragraph}"
            continue
        
        if paragraph_text.startswith("Mục"):
            current_section = paragraph_text
            current_mini_section = None
            continue
        
        if paragraph_text.startswith("Tiểu mục"):
            current_mini_section = paragraph_text
            continue
        
        if paragraph_text.startswith("Điều"):
            article = paragraph_text
            content = []
            
            for j in range(i + 1, len(document.paragraphs)):
                next_paragraph = document.paragraphs[j].text.strip()
                if next_paragraph.startswith(("Chương", "Mục", "Tiểu mục", "Điều")):
                    break
                content.append(next_paragraph)
            
            content_text = "\n".join(content)
            combine_article_content = f"{article}\n{content_text}"

            # Tạo từ khóa từ nội dung
            content_key_words = extract_keywords(combine_article_content)
            content_key_words = process_keywords(content_key_words)

            # Tạo từ khóa tổng hợp từ file metadata và nội dung
            combined_keywords = list(dict.fromkeys(file_metadata["key_words"] + content_key_words))
            
            # Xử lý từ khóa có ký tự "/"
            for kw in combined_keywords[:]:
                if "/" in kw:
                    new_kw = kw.replace("/", "-")
                    if new_kw not in combined_keywords:
                        combined_keywords.append(new_kw)

            # Tách nội dung thành từng phần dựa trên số thứ tự (1., 2., 3., ...)
            numbered_sections = re.split(r'(?=\d+\.\s)', content_text)

            part_index = 0  # Để đếm các phần
            current_content = ""
            for section in numbered_sections:
                if count_tokens_simple(current_content + section) <= max_tokens:
                    current_content += section
                else:
                    # Lưu phần nội dung hiện tại
                    data.append({
                        "Chapter": current_chapter,
                        "Chapter_Keywords": process_value_to_keywords(current_chapter),
                        "Section": current_section if current_section else None,
                        "Section_Keywords": process_value_to_keywords(current_section),
                        "Mini-Section": current_mini_section if current_mini_section else None,
                        "Mini-Section_Keywords": process_value_to_keywords(current_mini_section),
                        "Article": article,
                        "Article_Keywords": process_value_to_keywords(article),
                        "Content": current_content.strip(),
                        "Content_Keywords": process_value_to_keywords(current_content),
                        "combine_Article_Content": combine_article_content,
                        "key_words": combined_keywords
                    })
                    part_index += 1
                    current_content = section
            
            # Lưu phần nội dung còn lại nếu có
            if current_content:
                data.append({
                    "Chapter": current_chapter,
                    "Chapter_Keywords": process_value_to_keywords(current_chapter),
                    "Section": current_section if current_section else None,
                    "Section_Keywords": process_value_to_keywords(current_section),
                    "Mini-Section": current_mini_section if current_mini_section else None,
                    "Mini-Section_Keywords": process_value_to_keywords(current_mini_section),
                    "Article": article,
                    "Article_Keywords": process_value_to_keywords(article),
                    "Content": current_content.strip(),
                    "Content_Keywords": process_value_to_keywords(current_content),
                    "combine_Article_Content": combine_article_content,
                    "key_words": combined_keywords
                })

    return data

## 2.6. Hàm chính xử lý tất cả các file trong thư mục

In [8]:
def process_folder(folder_path, output_json_path, max_tokens):
    all_data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".docx"):
            file_metadata = process_file_name(file_name)
            if file_metadata is None:
                continue
            
            file_path = os.path.join(folder_path, file_name)
            docx_data = process_docx(file_path, file_metadata, max_tokens=max_tokens)
            
            for entry in docx_data:
                combined_entry = {**file_metadata, **entry}
                all_data.append(combined_entry)
    
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"Dữ liệu đã được lưu vào file {output_json_path}")

## 2.6. Thực thi

In [9]:
# Đường dẫn tới thư mục và file đầu ra
folder_path = r'data\trich_dan_luat\docx\Luat'
json_file_path = r'data\trich_dan_luat\json\test_Article_Combined_Output_More_Keywords.json'
max_tokens = 2048  # Giới hạn số token

In [10]:
# Chạy chương trình chính
process_folder(folder_path, json_file_path, max_tokens)

Dữ liệu đã được lưu vào file data\trich_dan_luat\json\test_Article_Combined_Output_More_Keywords.json
